<a href="https://colab.research.google.com/github/AliGhadirii/Fuzzy-Deep-Learning-Text-Classification/blob/main/Final_CHi_Sentiment_V7_3class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Set up Directory { run: "auto"}
import os
from IPython.display import clear_output
from google.colab import drive 
from IPython.display import clear_output
drive.mount('/content/gdrive')
working_directory = 'My Drive/SEN_BERT' #@param {type:"string"}
wd="/content/gdrive/"+working_directory
os.chdir(wd)

dirpath = os.getcwd()
print("current directory is : " + dirpath)


Mounted at /content/gdrive
current directory is : /content/gdrive/My Drive/SEN_BERT


#Training Model_1 : BERT_base

In [ ]:
! pip install transformers -q
! pip install emoji --upgrade -q
! pip install Unidecode -q

     |████████████████████████████████| 2.0MB 9.3MB/s 
     |████████████████████████████████| 890kB 48.4MB/s 
     |████████████████████████████████| 3.2MB 52.4MB/s 
     |████████████████████████████████| 133kB 7.5MB/s 
     |████████████████████████████████| 245kB 7.8MB/s 


In [ ]:
import re
import pandas as pd
from pathlib import Path
import matplotlib.cm as cm
import numpy as np
import pandas as pd
from typing import *
from tqdm.notebook import tqdm
from sklearn.utils.extmath import softmax
from sklearn import model_selection
from sklearn.metrics import classification_report, f1_score

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import transformers
from transformers import AdamW

In [ ]:
import emoji
import regex
import numpy as np
import os
import random

In [ ]:
def seed_all(seed = 42):
  """
  Fix seed for reproducibility
  """
  random.seed(seed)  
  torch.manual_seed(seed)
  torch.backends.cudnn.deterministic = True
  if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)
  np.random.seed(seed)

In [ ]:
class config:
  SEED = 42
  KFOLD = 5
  TRAIN_FILE = 'train.csv'
  VAL_FILE =  'valid.csv'
  TEST_FILE = 'test.csv'
  OOF_FILE = 'oof.csv'
  MAX_LEN = 40
  MODEL = 'bert-base-uncased'
  TOKENIZER = transformers.AutoTokenizer.from_pretrained(MODEL)
  EPOCHS = 5
  TRAIN_BATCH_SIZE = 16
  VALID_BATCH_SIZE = 16
  DEVICE = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")

In [ ]:
class AverageMeter:
    """
    Computes and stores the average and current value
    Source : https://www.kaggle.com/abhishek/bert-base-uncased-using-pytorch/
    """
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
class EarlyStopping:
    """
    Early stopping utility
    Source : https://www.kaggle.com/abhishek/bert-base-uncased-using-pytorch/
    """
    
    def __init__(self, patience=7, mode="max", delta=0.001):
        self.patience = patience
        self.counter = 0
        self.mode = mode
        self.best_score = None
        self.early_stop = False
        self.delta = delta
        if self.mode == "min":
            self.val_score = np.Inf
        else:
            self.val_score = -np.Inf

    def __call__(self, epoch_score, model, model_path):
        if self.mode == "min":
            score = -1.0 * epoch_score
        else:
            score = np.copy(epoch_score)
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(epoch_score, model, model_path)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print('EarlyStopping counter: {} out of {}'.format(self.counter, self.patience))
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(epoch_score, model, model_path)
            self.counter = 0

    def save_checkpoint(self, epoch_score, model, model_path):
        if epoch_score not in [-np.inf, np.inf, -np.nan, np.nan]:
            print('Validation score improved ({} --> {}). Saving model!'.format(self.val_score, epoch_score))
            torch.save(model.state_dict(), model_path)
        self.val_score = epoch_score

In [ ]:
def process_data(text, tokenizer, max_len, label):

  token_ids = tokenizer.encode(text, add_special_tokens=True)
  mask = [1] * len(token_ids)

  padding = max_len - len(token_ids)
  
  if padding>=0:
    token_ids = token_ids + ([0] * padding)
    mask = mask + ([0] * padding)
  else:
    token_ids = token_ids[0:max_len]
    mask = mask[0:max_len]

  assert len(token_ids)==max_len
  assert len(mask)==max_len

  return {'text':text,
          'ids':token_ids,
          'mask':mask,
          'label':label
          }

In [ ]:
class Dataset:
    def __init__(self, text, label):
        self.text = text
        self.label = label
        self.tokenizer = config.TOKENIZER
        self.max_len = config.MAX_LEN
    
    def __len__(self):
        return len(self.text)

    def __getitem__(self, item):
        data = process_data(
            self.text[item], 
            self.tokenizer,
            self.max_len,
            self.label[item],
        )

        return {
            'ids': torch.tensor(data["ids"], dtype=torch.long),
            'mask': torch.tensor(data["mask"], dtype=torch.long),
            'text': data['text'],
            'label': data['label'],
        }

In [ ]:
def train_fn(data_loader, model, optimizer, device):
  model.train()
  losses = AverageMeter()
  tk0 = tqdm(data_loader, total=len(data_loader))
  
  for bi, d in enumerate(tk0):
    ids = d['ids']
    mask = d['mask']
    label = d['label']

    ids = ids.to(device, dtype=torch.long)
    label = label.to(device, dtype=torch.long)
    mask = mask.to(device, dtype=torch.long)

    model.zero_grad()
    loss, logits = model(input_ids=ids, attention_mask=mask, labels=label)

    loss.backward()
    optimizer.step()

    losses.update(loss.item(), ids.size(0))
    tk0.set_postfix(loss=losses.avg)


In [ ]:
def eval_fn(data_loader, model, device):
  model.eval()
  losses = AverageMeter()
  tk0 = tqdm(data_loader, total=len(data_loader))
  yt, yp = [], []

  for bi, d in enumerate(tk0):
    ids = d['ids']
    mask = d['mask']
    label = d['label']

    ids = ids.to(device, dtype=torch.long)
    label = label.to(device, dtype=torch.long)
    mask = mask.to(device, dtype=torch.long)

    with torch.no_grad():
      loss, logits = model(input_ids=ids, attention_mask=mask, labels=label)        
       
    
    logits = logits.detach().cpu().numpy()

    preds = softmax(logits)
    pred_labels = np.argmax(preds, axis=1).flatten()
    ground_labels = label.to('cpu').numpy()

    yt = yt + ground_labels.tolist()
    yp = yp + pred_labels.tolist()

    losses.update(loss.item(), ids.size(0))
    tk0.set_postfix(loss=losses.avg)

  return f1_score(yt, yp, average='weighted')


In [ ]:
def test_fn(data_loader, model, device):
  model.eval()
  tk0 = tqdm(data_loader, total=len(data_loader))
  test_preds = []

  for bi, d in enumerate(tk0):
    ids = d['ids']
    mask = d['mask']
    label = d['label']
    
    ids = ids.to(device, dtype=torch.long)
    mask = mask.to(device, dtype=torch.long)
    label = label.to(device, dtype=torch.long)

    with torch.no_grad():
      _, logits = model(input_ids=ids, attention_mask=mask, labels=label)        
    
    logits = logits.detach().cpu().numpy()
    preds = softmax(logits)     
    test_preds = test_preds + preds.tolist()

  return test_preds

In [ ]:
def run(df_train, df_val, fold=None):
  train_dataset = Dataset(
        text = df_train.full_text.values,
        label = df_train.Sentiment_Label.values,
    )
  
  valid_dataset = Dataset(
      text = df_val.full_text.values,
      label = df_val.Sentiment_Label.values,
    )
  
  train_data_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=config.TRAIN_BATCH_SIZE,
        num_workers=4
    )

  valid_data_loader = torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=config.VALID_BATCH_SIZE,
        num_workers=2
    )
  
  
  model = transformers.BertForSequenceClassification.from_pretrained(config.MODEL, num_labels=3, return_dict=False)
  device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")
  model.to(device)

  lr = 2e-5
  param_optimizer = list(model.named_parameters())
  no_decay = ['bias', 'gamma', 'beta']
  optimizer_grouped_parameters = [
      {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
      'weight_decay_rate': 0.01},
      {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
      'weight_decay_rate': 0.0}
  ]
  optimizer = AdamW(optimizer_grouped_parameters, lr=lr)

  es = EarlyStopping(patience=2, mode="max")

  print('Starting training....')
  for epoch in range(config.EPOCHS):
    train_fn(train_data_loader, model, optimizer, device)
    valid_loss = eval_fn(valid_data_loader, model, device)
    print(f'Epoch :{epoch + 1} | Validation Score :{valid_loss}')
    if fold is None:
      es(valid_loss, model, model_path="model.bin")
    else:
      es(valid_loss, model, model_path=f"model_{fold}.bin")
    if es.early_stop:
      print('Early stopping')
      break

  print('Predicting for OOF')
  if fold is None:
    model.load_state_dict(torch.load('model.bin'))
  else:
    model.load_state_dict(torch.load(f'model_{fold}.bin'))
    
  model.to(device)
  
  test_predictions = test_fn(valid_data_loader, model, device)
  return test_predictions

In [ ]:
def run_fold(fold_idx):
  """
    Perform k-fold cross-validation
  """

  seed_all()
  scores = pd.DataFrame()

  df_train = pd.read_csv(config.TRAIN_FILE)
  df_val = pd.read_csv(config.VAL_FILE)
  
  # concatenating train and validation set
  train = pd.concat([df_train, df_val]).reset_index()
  
  # dividing folds
  kf = model_selection.StratifiedKFold(n_splits=config.KFOLD, shuffle=False, random_state=config.SEED)
  idx = None

  for fold, (train_idx, val_idx) in enumerate(kf.split(X=train, y=train.Sentiment_Label.values)):
      train.loc[val_idx, 'kfold'] = fold
      if fold==fold_idx:
        idx = val_idx

  if os.path.isfile(config.OOF_FILE):
    scores = pd.read_csv(config.OOF_FILE)
    print('Found oof file')
  else:
    scores = train.copy()
    scores['oof'] = 0
    scores.to_csv(config.OOF_FILE, index=False)
    print('Created oof file')
  
  df_train = train[train.kfold!=fold_idx]
  df_val = train[train.kfold==fold_idx]

  y = run(df_train, df_val, fold_idx)
  pred_label = np.argmax(y, axis=1).flatten()
  scores.loc[idx, 'oof'] = pred_label
  
  scores.to_csv(config.OOF_FILE, index=False)


In [ ]:
run_fold(0)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Created oof file


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or w

Starting training....



Epoch :1 | Validation Score :0.8076590163228912
Validation score improved (-inf --> 0.8076590163228912). Saving model!



Epoch :2 | Validation Score :0.8010700515759133
EarlyStopping counter: 1 out of 2



Epoch :3 | Validation Score :0.794082744697871
EarlyStopping counter: 2 out of 2
Early stopping
Predicting for OOF


In [ ]:
# df_val = pd.read_csv(config.VAL_FILE)

# valid_dataset = Dataset(
#     text = df_val.full_text.values,
#     label = df_val.Sentiment_Label.values,
#   )

# valid_data_loader = torch.utils.data.DataLoader(
#         valid_dataset,
#         batch_size=config.VALID_BATCH_SIZE,
#         num_workers=2
#     )

# model = transformers.BertForSequenceClassification.from_pretrained(config.MODEL, num_labels=3, return_dict=False)
# device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")
# model.load_state_dict(torch.load('model_0.bin'))
# model.to(device)
# test_predictions = test_fn(valid_data_loader, model, device)
# pred_label = np.argmax(test_predictions, axis=1).flatten()
# df_val['pred_label'] = pred_label
# temp = df_val[df_val['Sentiment_Label'] != df_val['pred_label']]
# temp
temp.to_csv('eval.csv', index=False)

In [ ]:
run_fold(1)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Found oof file


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Starting training....



Epoch :1 | Validation Score :0.807780723458744
Validation score improved (-inf --> 0.807780723458744). Saving model!



Epoch :2 | Validation Score :0.8066207483834981
EarlyStopping counter: 1 out of 2



Epoch :3 | Validation Score :0.7989863644625798
EarlyStopping counter: 2 out of 2
Early stopping
Predicting for OOF


In [ ]:
run_fold(2)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Found oof file


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Starting training....



Epoch :1 | Validation Score :0.807918600469505
Validation score improved (-inf --> 0.807918600469505). Saving model!



Epoch :2 | Validation Score :0.8082996499871115
EarlyStopping counter: 1 out of 2



Epoch :3 | Validation Score :0.7928176002077414
EarlyStopping counter: 2 out of 2
Early stopping
Predicting for OOF


In [ ]:
run_fold(3)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Found oof file


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Starting training....



Epoch :1 | Validation Score :0.7983176511665255
Validation score improved (-inf --> 0.7983176511665255). Saving model!



Epoch :2 | Validation Score :0.7897406285909349
EarlyStopping counter: 1 out of 2



Epoch :3 | Validation Score :0.7867248059618113
EarlyStopping counter: 2 out of 2
Early stopping
Predicting for OOF


In [ ]:
run_fold(4)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Found oof file


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Starting training....



Epoch :1 | Validation Score :0.7869780118917878
Validation score improved (-inf --> 0.7869780118917878). Saving model!



Epoch :2 | Validation Score :0.7953702890080961
Validation score improved (0.7869780118917878 --> 0.7953702890080961). Saving model!



Epoch :3 | Validation Score :0.7914818958542914
EarlyStopping counter: 1 out of 2



Epoch :4 | Validation Score :0.7799164920967608
EarlyStopping counter: 2 out of 2
Early stopping
Predicting for OOF


#Training Model_2 : CT_BERT

In [ ]:
os.chdir('../SEN_CTBERT')
print(os.getcwd())

/content/gdrive/My Drive/SEN_CTBERT


In [ ]:
! pip install transformers -q
! pip install emoji --upgrade -q
! pip install Unidecode -q

     |████████████████████████████████| 2.0MB 19.7MB/s 
     |████████████████████████████████| 890kB 25.7MB/s 
     |████████████████████████████████| 3.2MB 47.2MB/s 
     |████████████████████████████████| 133kB 17.5MB/s 
     |████████████████████████████████| 245kB 19.9MB/s 


In [ ]:
import re
import pandas as pd
from pathlib import Path
import matplotlib.cm as cm
import numpy as np
import pandas as pd
from typing import *
from tqdm.notebook import tqdm
from sklearn.utils.extmath import softmax
from sklearn import model_selection
from sklearn.metrics import classification_report, f1_score

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import transformers
from transformers import AdamW

In [ ]:
import emoji
import regex
import numpy as np
import os
import random

In [ ]:
def seed_all(seed = 42):
  """
  Fix seed for reproducibility
  """
  random.seed(seed)  
  torch.manual_seed(seed)
  torch.backends.cudnn.deterministic = True
  if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)
  np.random.seed(seed)

In [ ]:
class config:
  SEED = 42
  KFOLD = 5
  TRAIN_FILE = 'train.csv'
  VAL_FILE =  'valid.csv'
  TEST_FILE = 'test.csv'
  OOF_FILE = 'oof.csv'
  MAX_LEN = 40
  MODEL = 'digitalepidemiologylab/covid-twitter-bert'
  TOKENIZER = transformers.AutoTokenizer.from_pretrained(MODEL)
  EPOCHS = 5
  TRAIN_BATCH_SIZE = 16
  VALID_BATCH_SIZE = 16
  DEVICE = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")

In [ ]:
class AverageMeter:
    """
    Computes and stores the average and current value
    Source : https://www.kaggle.com/abhishek/bert-base-uncased-using-pytorch/
    """
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
class EarlyStopping:
    """
    Early stopping utility
    Source : https://www.kaggle.com/abhishek/bert-base-uncased-using-pytorch/
    """
    
    def __init__(self, patience=7, mode="max", delta=0.001):
        self.patience = patience
        self.counter = 0
        self.mode = mode
        self.best_score = None
        self.early_stop = False
        self.delta = delta
        if self.mode == "min":
            self.val_score = np.Inf
        else:
            self.val_score = -np.Inf

    def __call__(self, epoch_score, model, model_path):
        if self.mode == "min":
            score = -1.0 * epoch_score
        else:
            score = np.copy(epoch_score)
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(epoch_score, model, model_path)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print('EarlyStopping counter: {} out of {}'.format(self.counter, self.patience))
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(epoch_score, model, model_path)
            self.counter = 0

    def save_checkpoint(self, epoch_score, model, model_path):
        if epoch_score not in [-np.inf, np.inf, -np.nan, np.nan]:
            print('Validation score improved ({} --> {}). Saving model!'.format(self.val_score, epoch_score))
            torch.save(model.state_dict(), model_path)
        self.val_score = epoch_score

In [ ]:
def process_data(text, tokenizer, max_len, label):

  
  token_ids = tokenizer.encode(text, add_special_tokens=True)
  mask = [1] * len(token_ids)

  padding = max_len - len(token_ids)
  
  if padding>=0:
    token_ids = token_ids + ([0] * padding)
    mask = mask + ([0] * padding)
  else:
    token_ids = token_ids[0:max_len]
    mask = mask[0:max_len]

  
  assert len(token_ids)==max_len
  assert len(mask)==max_len

  return {'text':text,
          'ids':token_ids,
          'mask':mask,
          'label':label
          }

In [ ]:
class Dataset:
    def __init__(self, text, label):
        self.text = text
        self.label = label
        self.tokenizer = config.TOKENIZER
        self.max_len = config.MAX_LEN
    
    def __len__(self):
        return len(self.text)

    def __getitem__(self, item):
        data = process_data(
            self.text[item], 
            self.tokenizer,
            self.max_len,
            self.label[item],
        )

        return {
            'ids': torch.tensor(data["ids"], dtype=torch.long),
            'mask': torch.tensor(data["mask"], dtype=torch.long),
            'text': data['text'],
            'label': data['label'],
        }

In [ ]:
class FGM():
    """
        Utility for Adversarial Training
        Source: https://www.kaggle.com/c/tweet-sentiment-extraction/discussion/143764
    """
    def __init__(self, model):
        self.model = model
        self.backup = {}

    def attack(self, epsilon=1., emb_name='word_embeddings'):
        for name, param in self.model.named_parameters():
            if param.requires_grad and emb_name in name:
                self.backup[name] = param.data.clone()
                norm = torch.norm(param.grad)
                if norm != 0:
                    r_at = epsilon * param.grad / norm
                    param.data.add_(r_at)

    def restore(self, emb_name='word_embeddings'):
        for name, param in self.model.named_parameters():
            if param.requires_grad and emb_name in name:
                assert name in self.backup
                param.data = self.backup[name]
            self.backup = {}

In [ ]:
def train_fn(data_loader, model, optimizer, device):
  model.train()
  fgm = FGM(model)

  losses = AverageMeter()
  tk0 = tqdm(data_loader, total=len(data_loader))
  
  for bi, d in enumerate(tk0):
    ids = d['ids']
    mask = d['mask']
    label = d['label']

    ids = ids.to(device, dtype=torch.long)
    label = label.to(device, dtype=torch.long)
    mask = mask.to(device, dtype=torch.long)

    model.zero_grad()
    loss, logits = model(input_ids=ids, attention_mask=mask, labels=label)
    loss.backward()

    fgm.attack() 
    loss_adv, _ = model(input_ids=ids, attention_mask=mask, labels=label)
    loss_adv.backward() 
    fgm.restore()

    optimizer.step()

    losses.update(loss.item(), ids.size(0))
    tk0.set_postfix(loss=losses.avg)


In [ ]:
def eval_fn(data_loader, model, device):
  model.eval()
  losses = AverageMeter()
  tk0 = tqdm(data_loader, total=len(data_loader))
  yt, yp = [], []

  for bi, d in enumerate(tk0):
    ids = d['ids']
    mask = d['mask']
    label = d['label']

    ids = ids.to(device, dtype=torch.long)
    label = label.to(device, dtype=torch.long)
    mask = mask.to(device, dtype=torch.long)

    with torch.no_grad():
      loss, logits = model(input_ids=ids, attention_mask=mask, labels=label)        
       
    
    logits = logits.detach().cpu().numpy()

    preds = softmax(logits)
    pred_labels = np.argmax(preds, axis=1).flatten()
    ground_labels = label.to('cpu').numpy()

    yt = yt + ground_labels.tolist()
    yp = yp + pred_labels.tolist()

    losses.update(loss.item(), ids.size(0))
    tk0.set_postfix(loss=losses.avg)

  return f1_score(yt, yp, average='weighted')


In [ ]:
def test_fn(data_loader, model, device):
  model.eval()
  tk0 = tqdm(data_loader, total=len(data_loader))
  test_preds = []

  for bi, d in enumerate(tk0):
    ids = d['ids']
    mask = d['mask']
    label = d['label']
    
    ids = ids.to(device, dtype=torch.long)
    mask = mask.to(device, dtype=torch.long)
    label = label.to(device, dtype=torch.long)

    with torch.no_grad():
      _, logits = model(input_ids=ids, attention_mask=mask, labels=label)        
    
    logits = logits.detach().cpu().numpy()
    preds = softmax(logits)     
    test_preds = test_preds + preds.tolist()

  return test_preds

In [ ]:
def run(df_train, df_val, fold=None):
  train_dataset = Dataset(
        text = df_train.full_text.values,
        label = df_train.Sentiment_Label.values,
    )
  
  valid_dataset = Dataset(
      text = df_val.full_text.values,
      label = df_val.Sentiment_Label.values,
    )
  
  train_data_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=config.TRAIN_BATCH_SIZE,
        num_workers=4
    )

  valid_data_loader = torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=config.VALID_BATCH_SIZE,
        num_workers=2
    )
  
  
  model = transformers.BertForSequenceClassification.from_pretrained(config.MODEL, num_labels=3, return_dict=False)
  device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")
  model.to(device)

  lr = 2e-5
  param_optimizer = list(model.named_parameters())
  no_decay = ['bias', 'gamma', 'beta']
  optimizer_grouped_parameters = [
      {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
      'weight_decay_rate': 0.01},
      {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
      'weight_decay_rate': 0.0}
  ]
  optimizer = AdamW(optimizer_grouped_parameters, lr=lr)

  es = EarlyStopping(patience=2, mode="max")

  print('Starting training....')
  for epoch in range(config.EPOCHS):
    train_fn(train_data_loader, model, optimizer, device)
    valid_loss = eval_fn(valid_data_loader, model, device)
    print(f'Epoch :{epoch + 1} | Validation Score :{valid_loss}')
    if fold is None:
      es(valid_loss, model, model_path="model.bin")
    else:
      es(valid_loss, model, model_path=f"model_{fold}.bin")
    if es.early_stop:
      print('Early stopping')
      break

  print('Predicting for OOF')
  if fold is None:
    model.load_state_dict(torch.load('model.bin'))
  else:
    model.load_state_dict(torch.load(f'model_{fold}.bin'))
    
  model.to(device)
  
  test_predictions = test_fn(valid_data_loader, model, device)
  return test_predictions

In [ ]:
def run_fold(fold_idx):
  """
    Perform k-fold cross-validation
  """

  seed_all()
  scores = pd.DataFrame()

  df_train = pd.read_csv(config.TRAIN_FILE)
  df_val = pd.read_csv(config.VAL_FILE)
  
  # concatenating train and validation set
  train = pd.concat([df_train, df_val]).reset_index()
  
  # dividing folds
  kf = model_selection.StratifiedKFold(n_splits=config.KFOLD, shuffle=False, random_state=config.SEED)
  idx = None

  for fold, (train_idx, val_idx) in enumerate(kf.split(X=train, y=train.Sentiment_Label.values)):
      train.loc[val_idx, 'kfold'] = fold
      if fold==fold_idx:
        idx = val_idx

  if os.path.isfile(config.OOF_FILE):
    scores = pd.read_csv(config.OOF_FILE)
    print('Found oof file')
  else:
    scores = train.copy()
    scores['oof'] = 0
    scores.to_csv(config.OOF_FILE, index=False)
    print('Created oof file')
  
  df_train = train[train.kfold!=fold_idx]
  df_val = train[train.kfold==fold_idx]

  y = run(df_train, df_val, fold_idx)
  pred_label = np.argmax(y, axis=1).flatten()
  scores.loc[idx, 'oof'] = pred_label
  
  scores.to_csv(config.OOF_FILE, index=False)


In [ ]:
run_fold(0)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Found oof file


Some weights of the model checkpoint at digitalepidemiologylab/covid-twitter-bert were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassifi

Starting training....



Epoch :1 | Validation Score :0.8061287268133831
Validation score improved (-inf --> 0.8061287268133831). Saving model!



Epoch :2 | Validation Score :0.8047482531131762
EarlyStopping counter: 1 out of 2



Epoch :3 | Validation Score :0.7978145209717917
EarlyStopping counter: 2 out of 2
Early stopping
Predicting for OOF


In [ ]:
run_fold(1)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Found oof file


Some weights of the model checkpoint at digitalepidemiologylab/covid-twitter-bert were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassifi

Starting training....



Epoch :1 | Validation Score :0.8140717797774897
Validation score improved (-inf --> 0.8140717797774897). Saving model!



Epoch :2 | Validation Score :0.7996274913336086
EarlyStopping counter: 1 out of 2



Epoch :3 | Validation Score :0.8007758280255339
EarlyStopping counter: 2 out of 2
Early stopping
Predicting for OOF


In [ ]:
run_fold(2)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Found oof file


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Some weights of the model checkpoint at digitalepidemiologylab/covid-twitter-bert were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassifi

Starting training....



Epoch :1 | Validation Score :0.8076563808507622
Validation score improved (-inf --> 0.8076563808507622). Saving model!



Epoch :2 | Validation Score :0.809227234696961
Validation score improved (0.8076563808507622 --> 0.809227234696961). Saving model!



Epoch :3 | Validation Score :0.8001285077932871
EarlyStopping counter: 1 out of 2



Epoch :4 | Validation Score :0.7974775736973218
EarlyStopping counter: 2 out of 2
Early stopping
Predicting for OOF


In [ ]:
run_fold(3)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Found oof file


Some weights of the model checkpoint at digitalepidemiologylab/covid-twitter-bert were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassifi

Starting training....



Epoch :1 | Validation Score :0.8001855074938592
Validation score improved (-inf --> 0.8001855074938592). Saving model!


In [ ]:
run_fold(4)

NameError: ignored

#Training Model_3 : RoBERT

In [ ]:
! pip install transformers -q
! pip install emoji --upgrade -q
! pip install Unidecode -q

In [ ]:
import re
import pandas as pd
from pathlib import Path
import matplotlib.cm as cm
import numpy as np
import pandas as pd
from typing import *
from tqdm.notebook import tqdm
from sklearn.utils.extmath import softmax
from sklearn import model_selection
from sklearn.metrics import classification_report, f1_score

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import transformers
from transformers import AdamW

In [ ]:
import emoji
import os
import regex

In [ ]:
def seed_all(seed = 42):
  """
  Fix seed for reproducibility
  """
  # python RNG
  import random
  random.seed(seed)

  # pytorch RNGs
  import torch
  torch.manual_seed(seed)
  torch.backends.cudnn.deterministic = True
  if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

  # numpy RNG
  import numpy as np
  np.random.seed(seed)

In [ ]:
class config:
  SEED = 42
  KFOLD = 5
  
  TRAIN_FILE = 'train.csv'
  VAL_FILE =  'valid.csv'
  TEST_FILE = 'test.csv'
  OOF_FILE = 'oof.csv'
  MAX_LEN = 40
  MODEL = 'roberta-large'
  TOKENIZER = transformers.AutoTokenizer.from_pretrained(MODEL)
  EPOCHS = 5
  TRAIN_BATCH_SIZE = 16
  VALID_BATCH_SIZE = 16
  DEVICE = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")

In [ ]:
class AverageMeter:
    """
    Computes and stores the average and current value
    Source : https://www.kaggle.com/abhishek/bert-base-uncased-using-pytorch/
    """
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
class EarlyStopping:
    """
    Early stopping utility
    Source : https://www.kaggle.com/abhishek/bert-base-uncased-using-pytorch/
    """
    
    def __init__(self, patience=7, mode="max", delta=0.001):
        self.patience = patience
        self.counter = 0
        self.mode = mode
        self.best_score = None
        self.early_stop = False
        self.delta = delta
        if self.mode == "min":
            self.val_score = np.Inf
        else:
            self.val_score = -np.Inf

    def __call__(self, epoch_score, model, model_path):
        if self.mode == "min":
            score = -1.0 * epoch_score
        else:
            score = np.copy(epoch_score)
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(epoch_score, model, model_path)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print('EarlyStopping counter: {} out of {}'.format(self.counter, self.patience))
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(epoch_score, model, model_path)
            self.counter = 0

    def save_checkpoint(self, epoch_score, model, model_path):
        if epoch_score not in [-np.inf, np.inf, -np.nan, np.nan]:
            print('Validation score improved ({} --> {}). Saving model!'.format(self.val_score, epoch_score))
            torch.save(model.state_dict(), model_path)
        self.val_score = epoch_score

In [ ]:
def process_data(text, tokenizer, max_len, label):
  
  
  token_ids = tokenizer.encode(text, add_special_tokens=True)
  mask = [1] * len(token_ids)

  padding = max_len - len(token_ids)
  
  if padding>=0:
    token_ids = token_ids + ([0] * padding)
    mask = mask + ([0] * padding)
  else:
    token_ids = token_ids[0:max_len]
    mask = mask[0:max_len]



  assert len(token_ids)==max_len
  assert len(mask)==max_len

  return {'text':text,
          'ids':token_ids,
          'mask':mask,
          'label':label
          }

In [ ]:
class Dataset:
    def __init__(self, text, label):
        self.text = text
        self.label = label
        self.tokenizer = config.TOKENIZER
        self.max_len = config.MAX_LEN
    
    def __len__(self):
        return len(self.text)

    def __getitem__(self, item):
        data = process_data(
            self.text[item], 
            self.tokenizer,
            self.max_len,
            self.label[item],
        )

        return {
            'ids': torch.tensor(data["ids"], dtype=torch.long),
            'mask': torch.tensor(data["mask"], dtype=torch.long),
            'text': data['text'],
            'label': data['label'],
        }

In [ ]:
def train_fn(data_loader, model, optimizer, device):
  model.train()
  losses = AverageMeter()
  tk0 = tqdm(data_loader, total=len(data_loader))
  
  for bi, d in enumerate(tk0):
    ids = d['ids']
    mask = d['mask']
    label = d['label']

    ids = ids.to(device, dtype=torch.long)
    label = label.to(device, dtype=torch.long)
    mask = mask.to(device, dtype=torch.long)

    model.zero_grad()
    loss, logits = model(input_ids=ids, attention_mask=mask, labels=label)

    loss.backward()
    optimizer.step()

    losses.update(loss.item(), ids.size(0))
    tk0.set_postfix(loss=losses.avg)


In [ ]:
def eval_fn(data_loader, model, device):
  model.eval()
  losses = AverageMeter()
  tk0 = tqdm(data_loader, total=len(data_loader))
  yt, yp = [], []

  for bi, d in enumerate(tk0):
    ids = d['ids']
    mask = d['mask']
    label = d['label']

    ids = ids.to(device, dtype=torch.long)
    label = label.to(device, dtype=torch.long)
    mask = mask.to(device, dtype=torch.long)

    with torch.no_grad():
      loss, logits = model(input_ids=ids, attention_mask=mask, labels=label)        
       
    
    logits = logits.detach().cpu().numpy()

    preds = softmax(logits)
    pred_labels = np.argmax(preds, axis=1).flatten()
    ground_labels = label.to('cpu').numpy()

    yt = yt + ground_labels.tolist()
    yp = yp + pred_labels.tolist()

    losses.update(loss.item(), ids.size(0))
    tk0.set_postfix(loss=losses.avg)

  return f1_score(yt, yp)


In [ ]:
def run(df_train, df_val, fold=None):
  train_dataset = Dataset(
        text = df_train.Text.values,
        label = df_train.Label.values,
    )
  
  valid_dataset = Dataset(
      text = df_val.Text.values,
      label = df_val.Label.values,
    )
  
  train_data_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=config.TRAIN_BATCH_SIZE,
        num_workers=4
    )

  valid_data_loader = torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=config.VALID_BATCH_SIZE,
        num_workers=2
    )
  
  
  model = transformers.RobertaForSequenceClassification.from_pretrained(config.MODEL, num_labels=2, return_dict=False)
  device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")
  model.to(device)

  lr = 2e-5
  param_optimizer = list(model.named_parameters())
  no_decay = ['bias', 'gamma', 'beta']
  optimizer_grouped_parameters = [
      {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
      'weight_decay_rate': 0.01},
      {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
      'weight_decay_rate': 0.0}
  ]
  optimizer = AdamW(optimizer_grouped_parameters, lr=lr)

  es = EarlyStopping(patience=2, mode="max")

  print('Starting training....')
  for epoch in range(config.EPOCHS):
    train_fn(train_data_loader, model, optimizer, device)
    valid_loss = eval_fn(valid_data_loader, model, device)
    print(f'Epoch :{epoch + 1} | Validation Score :{valid_loss}')
    if fold is None:
      es(valid_loss, model, model_path=f"model.bin")
    else:
      es(valid_loss, model, model_path=f"model_{fold}.bin")
    if es.early_stop:
      print('Early stopping')
      break

  print('Predicting for OOF')
  if fold is None:
    model.load_state_dict(torch.load('model.bin'))
  else:
    model.load_state_dict(torch.load(f'model_{fold}.bin'))
    
  model.to(device)
  
  test_predictions = test_fn(valid_data_loader, model, device)
  return test_predictions

In [ ]:
def run_fold(fold_idx):
  """
    Perform k-fold cross-validation
  """

  seed_all()
  scores = pd.DataFrame()

  df_train = pd.read_csv(config.TRAIN_FILE)
  df_val = pd.read_csv(config.VAL_FILE)
  
  # concatenating train and validation set
  train = pd.concat([df_train, df_val]).reset_index()
  
  # dividing folds
  kf = model_selection.StratifiedKFold(n_splits=config.KFOLD, shuffle=False, random_state=config.SEED)
  idx = None

  for fold, (train_idx, val_idx) in enumerate(kf.split(X=train, y=train.Label.values)):
      train.loc[val_idx, 'kfold'] = fold
      if fold==fold_idx:
        idx = val_idx

  if os.path.isfile(config.OOF_FILE):
    scores = pd.read_csv(config.OOF_FILE)
    print('Found oof file')
  else:
    scores = train.copy()
    scores['oof'] = 0
    scores.to_csv(config.OOF_FILE, index=False)
    print('Created oof file')
  
  df_train = train[train.kfold!=fold_idx]
  df_val = train[train.kfold==fold_idx]

  y = run(df_train, df_val, fold_idx)
  scores.loc[idx, 'oof'] = y
  
  scores.to_csv(config.OOF_FILE, index=False)


In [ ]:
run_fold(0)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Found oof file


Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.weight', 'classif

Starting training....


Token indices sequence length is longer than the specified maximum sequence length for this model (1401 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (627 > 512). Running this sequence through the model will result in indexing errors


...


Epoch :4 | Validation Score :0.9560513860716701
EarlyStopping counter: 1 out of 2


Token indices sequence length is longer than the specified maximum sequence length for this model (1401 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (627 > 512). Running this sequence through the model will result in indexing errors


Token indices sequence length is longer than the specified maximum sequence length for this model (1035 > 512). Running this sequence through the model will result in indexing errors



Epoch :5 | Validation Score :0.9661129568106313
Validation score improved (0.9565217391304348 --> 0.9661129568106313). Saving model!
Predicting for OOF


Token indices sequence length is longer than the specified maximum sequence length for this model (1035 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
run_fold(1)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Found oof file


Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.weight', 'classif

Starting training....


Token indices sequence length is longer than the specified maximum sequence length for this model (1035 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1401 > 512). Running this sequence through the model will result in indexing errors


...


Epoch :2 | Validation Score :0.9609882964889467
Validation score improved (0.9550637156270958 --> 0.9609882964889467). Saving model!


Token indices sequence length is longer than the specified maximum sequence length for this model (1035 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1401 > 512). Running this sequence through the model will result in indexing errors



Epoch :3 | Validation Score :0.960797342192691
EarlyStopping counter: 1 out of 2


Token indices sequence length is longer than the specified maximum sequence length for this model (1035 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1401 > 512). Running this sequence through the model will result in indexing errors



Epoch :4 | Validation Score :0.9568679495686795
EarlyStopping counter: 2 out of 2
Early stopping
Predicting for OOF


In [ ]:
run_fold(2)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Found oof file


Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.weight', 'classif

Starting training....


Token indices sequence length is longer than the specified maximum sequence length for this model (1035 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1189 > 512). Running this sequence through the model will result in indexing errors


...


Epoch :4 | Validation Score :0.9572763684913218
Validation score improved (0.9554973821989529 --> 0.9572763684913218). Saving model!


Token indices sequence length is longer than the specified maximum sequence length for this model (1035 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1189 > 512). Running this sequence through the model will result in indexing errors


Token indices sequence length is longer than the specified maximum sequence length for this model (1401 > 512). Running this sequence through the model will result in indexing errors



Epoch :5 | Validation Score :0.9609164420485176
Validation score improved (0.9572763684913218 --> 0.9609164420485176). Saving model!
Predicting for OOF


Token indices sequence length is longer than the specified maximum sequence length for this model (1401 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
run_fold(3)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Found oof file


Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.weight', 'classif

Starting training....


Token indices sequence length is longer than the specified maximum sequence length for this model (1035 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1189 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1401 > 512). Running this sequence through the model will result in indexing errors


...


Epoch :4 | Validation Score :0.9433701657458563
EarlyStopping counter: 1 out of 2


Token indices sequence length is longer than the specified maximum sequence length for this model (1035 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1189 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1401 > 512). Running this sequence through the model will result in indexing errors


Token indices sequence length is longer than the specified maximum sequence length for this model (627 > 512). Running this sequence through the model will result in indexing errors



Epoch :5 | Validation Score :0.9569892473118279
Validation score improved (0.951058201058201 --> 0.9569892473118279). Saving model!
Predicting for OOF


Token indices sequence length is longer than the specified maximum sequence length for this model (627 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
run_fold(4)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Found oof file


Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.weight', 'classif

Starting training....


Token indices sequence length is longer than the specified maximum sequence length for this model (1035 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1189 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1401 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (627 > 512). Running this sequence through the model will result in indexing errors



Epoch :1 | Validation Score :0.9321692411014103
Validation score improved (-inf --> 0.9321692411014103). Saving model!


Token indices sequence length is longer than the specified maximum sequence length for this model (1035 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1189 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1401 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (627 > 512). Running this sequence through the model will result in indexing errors



Epoch :2 | Validation Score :0.9170013386880856
EarlyStopping counter: 1 out of 2


Token indices sequence length is longer than the specified maximum sequence length for this model (1035 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1189 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1401 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (627 > 512). Running this sequence through the model will result in indexing errors



Epoch :3 | Validation Score :0.9297297297297297
EarlyStopping counter: 2 out of 2
Early stopping
Predicting for OOF


#Train CHI

In [ ]:
import torch.nn.functional as F

def test_fn(data_loader, model, device):
  global num_data
  model.eval()
  tk0 = tqdm(data_loader, total=len(data_loader))
  test_preds = []

  temp_data = np.zeros([num_data, 1, 3])
  all_logits = []

  for bi, d in enumerate(tk0):
    ids = d['ids']
    mask = d['mask']
    label = d['label']
    
    ids = ids.to(device, dtype=torch.long)
    mask = mask.to(device, dtype=torch.long)
    label = label.to(device, dtype=torch.long)

    with torch.no_grad():
      _, logits = model(input_ids=ids, attention_mask=mask, labels=label)   
    all_logits.append(logits)
    
    logits = logits.detach().cpu().numpy()
    preds = softmax(logits)        
    test_preds = test_preds + preds.tolist()

  all_logits = torch.cat(all_logits, dim=0)
  probs = F.softmax(all_logits, dim=1).cpu().numpy()
  np.copyto(temp_data, probs.reshape(num_data, 1, 3))

  return np.array(test_preds), temp_data

In [ ]:
num_classes = 3

##Model_1

In [ ]:
threshold = 0.584
df = pd.read_csv(config.VAL_FILE)

val_dataset = Dataset(
      text = df.full_text.values,
      label = df.Sentiment_Label.values,
    )
  
val_data_loader = torch.utils.data.DataLoader(
      val_dataset,
      batch_size=config.VALID_BATCH_SIZE,
      num_workers=4
  )

num_data = len(df['Sentiment_Label'].values)
temp_train_data_1 = np.zeros([num_data, 1, num_classes])

scores = {}
T_datas = {}

model = transformers.BertForSequenceClassification.from_pretrained(config.MODEL, num_labels=3, return_dict=False)
device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")
model.to(device)

for i in range(config.KFOLD):
  model.load_state_dict(torch.load(f'model_{i}.bin'))
  y_logits, temp_train_data = test_fn(val_data_loader, model, device)
  scores[f'prob_{i}'] = y_logits
  T_datas[f'prob_{i}'] = temp_train_data

scores['avg'] = (scores['prob_0'] + scores['prob_1'] + scores['prob_2'] + scores['prob_3'] + scores['prob_4'])/5
scores['preds'] = np.argmax(scores['avg'], axis=1).flatten()

T_datas['data_avg'] = (T_datas['prob_0'] + T_datas['prob_1'] + T_datas['prob_2'] + T_datas['prob_3'] + T_datas['prob_4'])/5

np.copyto(temp_train_data_1, T_datas['data_avg'])

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
print(classification_report(scores['preds'], df['Sentiment_Label'].values, digits=4))
print(temp_train_data_1.shape)

              precision    recall  f1-score   support

           0     0.9230    0.9406    0.9317      2358
           1     0.7805    0.8000    0.7901      1000
           2     0.8902    0.8148    0.8508       826

    accuracy                         0.8822      4184
   macro avg     0.8646    0.8518    0.8576      4184
weighted avg     0.8825    0.8822    0.8819      4184

(4184, 1, 3)


##Model_2

In [ ]:
threshold = 0.584
os.chdir('../SEN_CTBERT')
df = pd.read_csv(config.VAL_FILE)

val_dataset = Dataset(
      text = df.full_text.values,
      label = df.Sentiment_Label.values,
    )
  
val_data_loader = torch.utils.data.DataLoader(
      val_dataset,
      batch_size=config.VALID_BATCH_SIZE,
      num_workers=4
  )


num_data = len(df['Sentiment_Label'].values)
temp_train_data_2 = np.zeros([num_data, 1, num_classes])

scores = {}
T_datas = {}

model = transformers.BertForSequenceClassification.from_pretrained(config.MODEL, num_labels=3, return_dict=False)
device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")
model.to(device)

for i in range(config.KFOLD):
  model.load_state_dict(torch.load(f'model_{i}.bin'))
  y_logits, temp_train_data = test_fn(val_data_loader, model, device)
  scores[f'prob_{i}'] = y_logits
  T_datas[f'prob_{i}'] = temp_train_data

scores['avg'] = (scores['prob_0'] + scores['prob_1'] + scores['prob_2'] + scores['prob_3'] + scores['prob_4'])/5
scores['preds'] = np.argmax(scores['avg'], axis=1).flatten()

T_datas['data_avg'] = (T_datas['prob_0'] + T_datas['prob_1'] + T_datas['prob_2'] + T_datas['prob_3'] + T_datas['prob_4'])/5

np.copyto(temp_train_data_2, T_datas['data_avg'])

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Some weights of the model checkpoint at digitalepidemiologylab/covid-twitter-bert were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassifi

In [ ]:
print(classification_report(scores['preds'], df['Sentiment_Label'].values, digits=4))
print(temp_train_data_2.shape)

              precision    recall  f1-score   support

           0     0.9605    0.9459    0.9531      2440
           1     0.8068    0.9009    0.8513       918
           2     0.9352    0.8559    0.8938       826

    accuracy                         0.9183      4184
   macro avg     0.9008    0.9009    0.8994      4184
weighted avg     0.9218    0.9183    0.9191      4184

(4184, 1, 3)


In [ ]:
scores['prob_0']

array([[6.89296305e-01, 2.66692132e-01, 4.40114848e-02],
       [1.60042018e-01, 6.42388344e-01, 1.97569624e-01],
       [1.35780284e-02, 7.47317016e-01, 2.39104912e-01],
       ...,
       [7.68298924e-01, 1.53609604e-01, 7.80914724e-02],
       [7.91968703e-01, 7.86571428e-02, 1.29374102e-01],
       [9.91314471e-01, 8.09644535e-03, 5.89133706e-04]])

##Model_3

In [ ]:
threshold = 0.584
os.chdir('../SEN_ROBERT')

df = pd.read_csv(config.VAL_FILE)

val_dataset = Dataset(
      text = df.full_text.values,
      label = df.Sentiment_Label.values,
    )
  
val_data_loader = torch.utils.data.DataLoader(
      val_dataset,
      batch_size=config.VALID_BATCH_SIZE,
      num_workers=4
  )

num_data = len(df['Sentiment_Label'].values)
temp_train_data_3 = np.zeros([num_data, 1, num_classes])

scores = {}
T_datas = {}


model = transformers.RobertaForSequenceClassification.from_pretrained(config.MODEL, num_labels=3, return_dict=False)
device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")
model.to(device)

for i in range(config.KFOLD):
  model.load_state_dict(torch.load(f'model_{i}.bin'))
  y_preds, temp_train_data = test_fn(val_data_loader, model, device)
  scores[f'prob_{i}'] = y_preds
  T_datas[f'prob_{i}'] = temp_train_data

scores['avg'] = (scores['prob_0'] + scores['prob_1'] + scores['prob_2'] + scores['prob_3'] + scores['prob_4'])/5
scores['preds'] = np.argmax(scores['avg'], axis=1).flatten()

T_datas['data_avg'] = (T_datas['prob_0'] + T_datas['prob_1'] + T_datas['prob_2'] + T_datas['prob_3'] + T_datas['prob_4'])/5

np.copyto(temp_train_data_3, T_datas['data_avg'])

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.weight', 'classif

In [ ]:
print(classification_report(scores['preds'], df['Sentiment_Label'].values, digits=4))
print(temp_train_data_3.shape)

              precision    recall  f1-score   support

           0     0.9392    0.9572    0.9481      2358
           1     0.8312    0.8580    0.8444       993
           2     0.9378    0.8511    0.8924       833

    accuracy                         0.9125      4184
   macro avg     0.9028    0.8888    0.8950      4184
weighted avg     0.9133    0.9125    0.9124      4184

(4184, 1, 3)


In [ ]:

#prepare train_data for CHI
train_data_CHI = np.concatenate((temp_train_data_1, temp_train_data_2, temp_train_data_3), axis=1)
train_label_CHI = df['Sentiment_Label'].to_numpy().astype('float64')
print(train_data_CHI.shape)


(4184, 3, 3)


In [ ]:
print('--Training--')
##############################################
# Train the ChI(s)
##############################################

import ChI
import cvxopt

CHIs = []
for j in range(0, num_classes):
    CHIs.append(ChI.ChoquetIntegral())
      
for j, chi in enumerate(CHIs):
    print('Class ChI {}'.format(j))
    train_data = np.transpose(train_data_CHI[:,:,j])
    label_data = train_label_CHI
    tr = chi.train_chi_quad(train_data, label_data)
    print(chi.fm)


--Training--
Class ChI 0
Number Inputs :  3 ; Number Samples :  4184
     pcost       dcost       gap    pres   dres
 0:  1.0625e+03  1.1824e+03  5e+02  2e+01  4e-02
 1:  1.1575e+03  1.2411e+03  1e+02  7e+00  1e-02
 2:  1.3506e+03  1.3012e+03  5e+01  7e-02  1e-04
 3:  1.3172e+03  1.3103e+03  7e+00  7e-04  1e-06
 4:  1.3149e+03  1.3147e+03  2e-01  2e-05  3e-08
 5:  1.3148e+03  1.3148e+03  2e-03  2e-07  3e-10
 6:  1.3148e+03  1.3148e+03  2e-05  2e-09  3e-12
Optimal solution found.
{'[1]': 1.6750696062319595e-06, '[2]': 0.41836569657901873, '[3]': 7.058645783591848e-08, '[1 2]': 0.9999999855555167, '[1 3]': 1.7244449824183912e-06, '[2 3]': 0.4183657474449637, '[1 2 3]': 1.0}
Class ChI 1
Number Inputs :  3 ; Number Samples :  4184
     pcost       dcost       gap    pres   dres
 0: -1.3221e+03 -1.3034e+03  2e+02  1e+01  8e-03
 1: -1.2888e+03 -1.2549e+03  6e+01  4e+00  3e-03
 2: -1.2509e+03 -1.2064e+03  5e+01  2e+00  1e-03
 3: -1.1990e+03 -1.1937e+03  5e+00  2e-01  1e-04
 4: -1.1924e+03 -1.

# Test the Hole model using CHI

In [ ]:
# train using densities
CHIs = []
densities=[0.8822, 0.9183, 0.9125]
densities = np.asarray(densities)

for j in range(0, num_classes):
    CHIs.append(ChI.ChoquetIntegral())
      
for j, chi in enumerate(CHIs):
    print('Class ChI {}'.format(j))
    tr = chi.train_chi_sugeno(densities)
    print(chi.fm)

Class ChI 0
[0.8822     0.9183     0.99107528 0.9125     0.99038762 0.99357482
 1.        ]
Class ChI 1
[0.8822     0.9183     0.99107528 0.9125     0.99038762 0.99357482
 1.        ]
Class ChI 2
[0.8822     0.9183     0.99107528 0.9125     0.99038762 0.99357482
 1.        ]


##Model_1

In [ ]:
threshold = 0.584
os.chdir('../SEN_BERT')
df = pd.read_csv(config.TEST_FILE)

test_dataset = Dataset(
      text = df.full_text.values,
      label = df.Sentiment_Label.values,
    )
  
test_data_loader = torch.utils.data.DataLoader(
      test_dataset,
      batch_size=config.VALID_BATCH_SIZE,
      num_workers=4
  )

num_data = len(df['Sentiment_Label'].values)
temp_test_data_1 = np.zeros([num_data, 1, num_classes])

scores = {}
T_datas = {}

model = transformers.BertForSequenceClassification.from_pretrained(config.MODEL, num_labels=3, return_dict=False)
device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")
model.to(device)

for i in range(config.KFOLD):
  model.load_state_dict(torch.load(f'model_{i}.bin'))
  y_preds, temp_test_data = test_fn(test_data_loader, model, device)
  scores[f'prob_{i}'] = y_preds
  T_datas[f'prob_{i}'] = temp_test_data

scores['avg'] = (scores['prob_0'] + scores['prob_1'] + scores['prob_2'] + scores['prob_3'] + scores['prob_4'])/5
scores['preds'] = np.argmax(scores['avg'], axis=1).flatten()

T_datas['data_avg'] = (T_datas['prob_0'] + T_datas['prob_1'] + T_datas['prob_2'] + T_datas['prob_3'] + T_datas['prob_4'])/5

np.copyto(temp_test_data_1, T_datas['data_avg'])

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or w

In [ ]:
print(classification_report(scores['preds'], df['Sentiment_Label'].values, digits=4))

              precision    recall  f1-score   support

           0     0.8935    0.8728    0.8830      2460
           1     0.5961    0.7197    0.6521       849
           2     0.8347    0.7211    0.7738       875

    accuracy                         0.8100      4184
   macro avg     0.7747    0.7712    0.7696      4184
weighted avg     0.8208    0.8100    0.8133      4184



##Model_2

In [ ]:
threshold = 0.584
os.chdir('../SEN_CTBERT')
df = pd.read_csv(config.TEST_FILE)

test_dataset = Dataset(
      text = df.full_text.values,
      label = df.Sentiment_Label.values,
    )
  
test_data_loader = torch.utils.data.DataLoader(
      test_dataset,
      batch_size=config.VALID_BATCH_SIZE,
      num_workers=4
  )

num_data = len(df['Sentiment_Label'].values)
temp_test_data_2 = np.zeros([num_data, 1, num_classes])

scores = {}
T_datas = {}

model = transformers.BertForSequenceClassification.from_pretrained(config.MODEL, num_labels=3, return_dict=False)
device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")
model.to(device)

for i in range(config.KFOLD):
  model.load_state_dict(torch.load(f'model_{i}.bin'))
  y_preds, temp_test_data = test_fn(test_data_loader, model, device)
  scores[f'prob_{i}'] = y_preds
  T_datas[f'prob_{i}'] = temp_test_data

scores['avg'] = (scores['prob_0'] + scores['prob_1'] + scores['prob_2'] + scores['prob_3'] + scores['prob_4'])/5
scores['preds'] = np.argmax(scores['avg'], axis=1).flatten()

T_datas['data_avg'] = (T_datas['prob_0'] + T_datas['prob_1'] + T_datas['prob_2'] + T_datas['prob_3'] + T_datas['prob_4'])/5

np.copyto(temp_test_data_2, T_datas['data_avg'])

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Some weights of the model checkpoint at digitalepidemiologylab/covid-twitter-bert were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification fro

In [ ]:
print(classification_report(scores['preds'], df['Sentiment_Label'].values, digits=4))

              precision    recall  f1-score   support

           0     0.8935    0.8728    0.8830      2460
           1     0.5961    0.7197    0.6521       849
           2     0.8347    0.7211    0.7738       875

    accuracy                         0.8100      4184
   macro avg     0.7747    0.7712    0.7696      4184
weighted avg     0.8208    0.8100    0.8133      4184



##Model_3

In [ ]:
threshold = 0.584
os.chdir('../SEN_ROBERT')

df = pd.read_csv(config.TEST_FILE)

test_dataset = Dataset(
      text = df.full_text.values,
      label = df.Sentiment_Label.values,
    )
  
test_data_loader = torch.utils.data.DataLoader(
      test_dataset,
      batch_size=config.VALID_BATCH_SIZE,
      num_workers=4
  )
num_data = len(df['Sentiment_Label'].values)
temp_test_data_3 = np.zeros([num_data, 1, num_classes])

scores = {}
T_datas = {}


model = transformers.RobertaForSequenceClassification.from_pretrained(config.MODEL, num_labels=3, return_dict=False)
device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")
model.to(device)

for i in range(config.KFOLD):
  model.load_state_dict(torch.load(f'model_{i}.bin'))
  y_preds, temp_test_data = test_fn(test_data_loader, model, device)
  scores[f'prob_{i}'] = y_preds
  T_datas[f'prob_{i}'] = temp_test_data

scores['avg'] = (scores['prob_0'] + scores['prob_1'] + scores['prob_2'] + scores['prob_3'] + scores['prob_4'])/5
scores['preds'] = np.argmax(scores['avg'], axis=1).flatten()

T_datas['data_avg'] = (T_datas['prob_0'] + T_datas['prob_1'] + T_datas['prob_2'] + T_datas['prob_3'] + T_datas['prob_4'])/5

np.copyto(temp_test_data_3, T_datas['data_avg'])

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model fr

In [ ]:
print(classification_report(scores['preds'], df['Sentiment_Label'].values, digits=4))

              precision    recall  f1-score   support

           0     0.8760    0.8904    0.8832      2364
           1     0.6390    0.6976    0.6670       939
           2     0.8373    0.7185    0.7734       881

    accuracy                         0.8109      4184
   macro avg     0.7841    0.7688    0.7745      4184
weighted avg     0.8147    0.8109    0.8115      4184



In [ ]:

#prepare train_data for CHI
test_data_CHI = np.concatenate((temp_test_data_1, temp_test_data_2, temp_test_data_3), axis=1)
test_label_CHI = df['Sentiment_Label'].to_numpy().astype('float64')
print(test_data_CHI.shape)


(4184, 3, 3)


In [ ]:
print('--Testing--')
##############################################
# Test the ChI(s)
##############################################
exper_out, known_out = [], []

for j in range(0, test_data_CHI.shape[0]): # for each data point
    out = []
    for k, chi in enumerate(CHIs): # for each ChI
        t_data_CHI = np.transpose(test_data_CHI[j, :, k])
        t_label = test_label_CHI[j]
        out.append(chi.chi_quad(t_data_CHI))
    out = np.asarray(out)
    exper_out.append(np.argmax(out))
    known_out.append(t_label)

--Testing--


In [ ]:
print('--Testing--')
##############################################
# Test the ChI(s) using densities
##############################################
exper_out, known_out = [], []

for j in range(0, test_data_CHI.shape[0]): # for each data point
    out = []
    for k, chi in enumerate(CHIs): # for each ChI
        t_data_CHI = np.transpose(test_data_CHI[j, :, k])
        t_label = test_label_CHI[j]
        out.append(chi.chi_sugeno(t_data_CHI))
    out = np.asarray(out)
    exper_out.append(np.argmax(out))
    known_out.append(t_label)

In [ ]:
# Evaluate the Bert classifier using CHI
print(classification_report(exper_out, known_out, digits=4))

              precision    recall  f1-score   support

           0     0.8856    0.8874    0.8865      2398
           1     0.6351    0.7092    0.6701       918
           2     0.8413    0.7327    0.7833       868

    accuracy                         0.8162      4184
   macro avg     0.7873    0.7764    0.7799      4184
weighted avg     0.8214    0.8162    0.8176      4184

